In [1]:
import sys
from google.colab import drive
drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/ByteTrack/')

Mounted at /content/gdrive/


In [2]:
cd '/content/gdrive/MyDrive/Colab Notebooks/ByteTrack/'

/content/gdrive/MyDrive/Colab Notebooks/ByteTrack


In [3]:
!pip3 install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.1 MB/s 
     |████████████████████████████████| 108 kB 10.6 MB/s 
     |████████████████████████████████| 1.5 MB 54.0 MB/s 
     |████████████████████████████████| 161 kB 53.6 MB/s 
     |████████████████████████████████| 177 kB 57.6 MB/s 
     |████████████████████████████████| 41 kB 247 kB/s 
     |████████████████████████████████| 1.9 MB 48.7 MB/s 
     |████████████████████████████████| 9.1 MB 46.2 MB/s 
     |████████████████████████████████| 873 kB 47.1 MB/s 
     |████████████████████████████████| 1.1 MB 49.7 MB/s 
     |████████████████████████████████| 745 kB 58.1 MB/s 
     |████████████████████████████████| 14.5 MB 42.6 MB/s 
     |████████████████████████████████| 4.5 MB 50.7 MB/s 
     |████████████████████████████████| 4.1 MB 20.8 MB/s 
     |████████████████████████████████| 28.2 MB 1.4 MB/s 
     |████████████████████████████

In [4]:
import os
import time
import argparse
import os.path as osp

import cv2
import torch
import matplotlib.pyplot as plt

from yolox.data.data_augment import preproc
from yolox.exp import get_exp
from yolox.utils import fuse_model, get_model_info, postprocess
from yolox.utils.visualize import plot_tracking
from yolox.tracker.byte_tracker import BYTETracker
from yolox.tracking_utils.timer import Timer

from IPython.display import clear_output
from time import sleep

In [5]:
# img = cv2.imread('./assets/test.jpg')

# plt.imshow(img)
# plt.axis('off')
# plt.show()

# height, width = img.shape[:2]
# print(height, width)

# num_classes = exp.num_classes
# confthre = exp.test_conf
# nmsthre = exp.nmsthre
# test_size = exp.test_size

# rgb_means = (0.485, 0.456, 0.406)
# std = (0.229, 0.224, 0.225)
# img, ratio = preproc(img, test_size, rgb_means, std)
# img = torch.from_numpy(img).unsqueeze(0).float().to(device)

# print(img.shape, ratio)

In [6]:
# exp = get_exp('exps/example/mot/yolox_x_mix_det.py', None)
exp = get_exp('exps/example/mot/yolox_s_mix_det.py', None)

def make_parser():
    parser = argparse.ArgumentParser("ByteTrack Demo!")

    parser.add_argument("--demo", default="webcam", help="demo type, eg. image, video and webcam")
    parser.add_argument("-expn", "--experiment-name", type=str, default='ByteTracker')
    parser.add_argument("-n", "--name", type=str, default='YOLOX', help="model name")
    
    parser.add_argument(
        #"--path", default="./datasets/mot/train/MOT17-05-FRCNN/img1", help="path to images or video"
        # "--path", default="./videos/palace.mp4", help="path to images or video"
        "--path", default="./assets/test.jpg", help="path to images or video"
    )
    
    parser.add_argument("--camid", type=int, default=0, help="webcam demo camera id")
    # parser.add_argument("--camid", type=str, default='https://192.168.1.253:4343/video', help="webcam demo camera id")
    parser.add_argument("--save_result", action="store_true", default='pretrained/', help="whether to save the inference result of image/video")
    
    # exp file
    parser.add_argument("-f", "--exp_file", default='yolox/exps/example/mot/yolox_s_mix_det.py', type=str, help="pls input your expriment description file")
    parser.add_argument("-c", "--ckpt", default='pretrained/bytetrack_s_mot17.pth.tar', type=str, help="ckpt for eval")
    
    parser.add_argument("--device", default="cpu", type=str, help="device to run our model, can either be cpu or gpu")
    parser.add_argument("--conf", default=exp.test_conf, type=float, help="test conf")
    parser.add_argument("--nms", default=exp.nmsthre, type=float, help="test nms threshold")
    parser.add_argument("--tsize", default=None, type=int, help="test img size")
    parser.add_argument("--fps", default=30, type=int, help="frame rate (fps)")
    parser.add_argument("--fp16", dest="fp16", default=False, action="store_true", help="Adopting mix precision evaluating.")

    parser.add_argument("--fuse", dest="fuse", default=False, action="store_true", help="Fuse conv and bn for testing.")
    parser.add_argument("--trt", dest="trt", default=False, action="store_true", help="Using TensorRT model for testing.")
    
    # tracking args
    parser.add_argument("--track_thresh", type=float, default=0.6, help="tracking confidence threshold")
    parser.add_argument("--track_buffer", type=int, default=30, help="the frames for keep lost tracks")
    parser.add_argument("--match_thresh", type=float, default=0.8, help="matching threshold for tracking")
    parser.add_argument("--aspect_ratio_thresh", type=float, default=1.6, help="threshold for filtering out boxes of which aspect ratio are above the given value.")
    parser.add_argument('--min_box_area', type=float, default=10, help='filter out tiny boxes')
    parser.add_argument("--mot20", dest="mot20", default=False, action="store_true", help="test mot20.")
    return parser

class Predictor(object):
    def __init__(self, model, exp, trt_file=None, decoder=None, device=torch.device("cpu"), fp16=False):
        self.model = model
        self.decoder = decoder
        self.num_classes = exp.num_classes
        self.confthre = exp.test_conf
        self.nmsthre = exp.nmsthre
        self.test_size = exp.test_size
        self.device = device
        self.fp16 = fp16
        
        if trt_file is not None:
            from torch2trt import TRTModule
            
            model_trt = TRTModule()
            model_trt.load_state_dict(torch.load(trt_file))

            x = torch.ones((1, 3, exp.test_size[0], exp.test_size[1]), device=device)
            self.model(x)
            self.model = model_trt
        self.rgb_means = (0.485, 0.456, 0.406)
        self.std = (0.229, 0.224, 0.225)

    def inference(self, img, timer):
        img_info = {"id": 0}
        if isinstance(img, str):
            img_info["file_name"] = osp.basename(img)
            img = cv2.imread(img)
        else:
            img_info["file_name"] = None
        
        height, width = img.shape[:2]
        img_info["height"] = height
        img_info["width"] = width
        img_info["raw_img"] = img

        img, ratio = preproc(img, self.test_size, self.rgb_means, self.std)
        img_info["ratio"] = ratio
        img = torch.from_numpy(img).unsqueeze(0).float().to(self.device)
        if self.fp16:
            img = img.half()  # to FP16
        
        with torch.no_grad():
            timer.tic()
            outputs = self.model(img)
            if self.decoder is not None:
                outputs = self.decoder(outputs, dtype=outputs.type())
            outputs = postprocess(outputs, self.num_classes, self.confthre, self.nmsthre)

            #logger.info("Infer time: {:.4f}s".format(time.time() - t0))
        return outputs, img_info

def get_image_list(path):
    image_names = []
    for maindir, subdir, file_name_list in os.walk(path):
        for filename in file_name_list:
            apath = osp.join(maindir, filename)
            ext = osp.splitext(apath)[1]
            if ext in IMAGE_EXT:
                image_names.append(apath)
    return image_names

def image_demo(predictor, vis_folder, current_time, args):
    if osp.isdir(args.path):
        files = get_image_list(args.path)
    else:
        files = [args.path]
    files.sort()
    tracker = BYTETracker(args, frame_rate=args.fps)
    timer = Timer()
    results = []

    for frame_id, img_path in enumerate(files, 1):
        outputs, img_info = predictor.inference(img_path, timer)
        if outputs[0] is not None:
            online_targets = tracker.update(outputs[0], [img_info['height'], img_info['width']], exp.test_size)
            online_tlwhs = []
            online_ids = []
            online_scores = []
            for t in online_targets:
                tlwh = t.tlwh
                tid = t.track_id
                vertical = tlwh[2] / tlwh[3] > args.aspect_ratio_thresh
                if tlwh[2] * tlwh[3] > args.min_box_area and not vertical:
                    online_tlwhs.append(tlwh)
                    online_ids.append(tid)
                    online_scores.append(t.score)
                    # save results
                    results.append(
                        f"{frame_id},{tid},{tlwh[0]:.2f},{tlwh[1]:.2f},{tlwh[2]:.2f},{tlwh[3]:.2f},{t.score:.2f},-1,-1,-1\n"
                    )
            timer.toc()
            online_im = plot_tracking(
                img_info['raw_img'], online_tlwhs, online_ids, frame_id=frame_id, fps=1. / timer.average_time
            )
        else:
            timer.toc()
            online_im = img_info['raw_img']

        # result_image = predictor.visual(outputs[0], img_info, predictor.confthre)
        if args.save_result:
            timestamp = time.strftime("%Y_%m_%d_%H_%M_%S", current_time)
            save_folder = osp.join(vis_folder, timestamp)
            os.makedirs(save_folder, exist_ok=True)
            cv2.imwrite(osp.join(save_folder, osp.basename(img_path)), online_im)

        if frame_id % 20 == 0:
            print('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))
        
        ch = cv2.waitKey(0)
        if ch == 27 or ch == ord("q") or ch == ord("Q"):
            break

    if args.save_result:
        res_file = osp.join(vis_folder, f"{timestamp}.txt")
        with open(res_file, 'w') as f:
            f.writelines(results)
        print(f"save results to {res_file}")

In [7]:
def imageflow_demo(predictor, vis_folder, current_time, args):
    cap = cv2.VideoCapture(args.path if args.demo == "video" else args.camid)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fps = cap.get(cv2.CAP_PROP_FPS)
    timestamp = time.strftime("%Y_%m_%d_%H_%M_%S", current_time)
    save_folder = osp.join(vis_folder, timestamp)
    os.makedirs(save_folder, exist_ok=True)
    if args.demo == "video":
        save_path = osp.join(save_folder, args.path.split("/")[-1])
    else:
        save_path = osp.join(save_folder, "camera.mp4")
    print(f"video save_path is {save_path}")
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (int(width), int(height)))
    tracker = BYTETracker(args, frame_rate=30)
    timer = Timer()
    frame_id = 0
    results = []
    while True:
        if frame_id % 20 == 0:
            print('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))
        ret_val, frame = cap.read()
        if ret_val:
            outputs, img_info = predictor.inference(frame, timer)
            if outputs[0] is not None:
                online_targets = tracker.update(outputs[0], [img_info['height'], img_info['width']], exp.test_size)
                online_tlwhs = []
                online_ids = []
                online_scores = []
                for t in online_targets:
                    tlwh = t.tlwh
                    tid = t.track_id
                    vertical = tlwh[2] / tlwh[3] > args.aspect_ratio_thresh
                    if tlwh[2] * tlwh[3] > args.min_box_area and not vertical:
                        online_tlwhs.append(tlwh)
                        online_ids.append(tid)
                        online_scores.append(t.score)
                        results.append(
                            f"{frame_id},{tid},{tlwh[0]:.2f},{tlwh[1]:.2f},{tlwh[2]:.2f},{tlwh[3]:.2f},{t.score:.2f},-1,-1,-1\n"
                        )
                timer.toc()
                online_im = plot_tracking(
                    img_info['raw_img'], online_tlwhs, online_ids, frame_id=frame_id + 1, fps=1. / timer.average_time
                )
            else:
                timer.toc()
                online_im = img_info['raw_img']
            if args.save_result:
                vid_writer.write(online_im)
            ch = cv2.waitKey(1)
            if ch == 27 or ch == ord("q") or ch == ord("Q"):
                break
        else:
            break
        frame_id += 1

    if args.save_result:
        res_file = osp.join(vis_folder, f"{timestamp}.txt")
        with open(res_file, 'w') as f:
            f.writelines(results)
        print(f"save results to {res_file}")

In [8]:
args = make_parser().parse_args()

In [9]:
# def main(exp, args):
#     if not args.experiment_name:
#         args.experiment_name = exp.exp_name

#     output_dir = osp.join(exp.output_dir, args.experiment_name)
#     os.makedirs(output_dir, exist_ok=True)
    
#     if args.save_result:
#         vis_folder = osp.join(output_dir, "track_vis")
#         os.makedirs(vis_folder, exist_ok=True)
    
#     if args.trt:
#         args.device = "gpu"
#     args.device = torch.device("cuda" if args.device == "gpu" else "cpu")
    
#     print("Args: {}".format(args))

#     if args.conf is not None:
#         exp.test_conf = args.conf
#     if args.nms is not None:
#         exp.nmsthre = args.nms
#     if args.tsize is not None:
#         exp.test_size = (args.tsize, args.tsize)

#     model = exp.get_model().to(args.device)
#     # print("Model Summary: {}".format(get_model_info(model, exp.test_size)))
#     model.eval()

#     if not args.trt:
#         if args.ckpt is None:
#             ckpt_file = osp.join(output_dir, "best_ckpt.pth.tar")
#         else:
#             ckpt_file = args.ckpt
#         print("loading checkpoint")
#         ckpt = torch.load(ckpt_file, map_location="cpu")
#         # load the model state dict
#         model.load_state_dict(ckpt["model"])
#         print("loaded checkpoint done.")

#     if args.fuse:
#         print("\tFusing model...")
#         model = fuse_model(model)

#     if args.fp16:
#         model = model.half()  # to FP16

#     if args.trt:
#         assert not args.fuse, "TensorRT model is not support model fusing!"
#         trt_file = osp.join(output_dir, "model_trt.pth")
#         assert osp.exists(
#             trt_file
#         ), "TensorRT model is not found!\n Run python3 tools/trt.py first!"
#         model.head.decode_in_inference = False
#         decoder = model.head.decode_outputs
#         print("Using TensorRT to inference")
#     else:
#         trt_file = None
#         decoder = None

#     predictor = Predictor(model, exp, trt_file, decoder, args.device, args.fp16)
#     current_time = time.localtime()
#     if args.demo == "image":
#         image_demo(predictor, vis_folder, current_time, args)
#     elif args.demo == "video" or args.demo == "webcam":
#         imageflow_demo(predictor, vis_folder, current_time, args)

# main(exp, args)

In [10]:
if not args.experiment_name:
    args.experiment_name = exp.exp_name

output_dir = osp.join(exp.output_dir, args.experiment_name)
os.makedirs(output_dir, exist_ok=True)

if args.save_result:
    vis_folder = osp.join(output_dir, "track_vis")
    os.makedirs(vis_folder, exist_ok=True)

if args.trt:
    args.device = "gpu"
args.device = torch.device("cuda" if args.device == "gpu" else "cpu")

print("Args: {}".format(args))

if args.conf is not None:
    exp.test_conf = args.conf
if args.nms is not None:
    exp.nmsthre = args.nms
if args.tsize is not None:
    exp.test_size = (args.tsize, args.tsize)

model = exp.get_model().to(args.device)
# print("Model Summary: {}".format(get_model_info(model, exp.test_size)))
model.eval()

if not args.trt:
    if args.ckpt is None:
        ckpt_file = osp.join(output_dir, "best_ckpt.pth.tar")
    else:
        ckpt_file = args.ckpt
    print("loading checkpoint")
    ckpt = torch.load(ckpt_file, map_location="cpu")
    # load the model state dict
    model.load_state_dict(ckpt["model"])
    print("loaded checkpoint done.")

if args.fuse:
    print("\tFusing model...")
    model = fuse_model(model)

if args.fp16:
    model = model.half()  # to FP16

if args.trt:
    assert not args.fuse, "TensorRT model is not support model fusing!"
    trt_file = osp.join(output_dir, "model_trt.pth")
    assert osp.exists(
        trt_file
    ), "TensorRT model is not found!\n Run python3 tools/trt.py first!"
    model.head.decode_in_inference = False
    decoder = model.head.decode_outputs
    print("Using TensorRT to inference")
else:
    trt_file = None
    decoder = None

predictor = Predictor(model, exp, trt_file, decoder, args.device, args.fp16)
current_time = time.localtime()

Args: Namespace(aspect_ratio_thresh=1.6, camid=0, ckpt='pretrained/bytetrack_s_mot17.pth.tar', conf=0.001, demo='webcam', device=device(type='cpu'), exp_file='/root/.local/share/jupyter/runtime/kernel-f6ef1469-6cdf-403b-9d5f-cfc4fbf188b9.json', experiment_name='ByteTracker', fp16=False, fps=30, fuse=False, match_thresh=0.8, min_box_area=10, mot20=False, name='YOLOX', nms=0.7, path='./assets/test.jpg', save_result='pretrained/', track_buffer=30, track_thresh=0.6, trt=False, tsize=None)
loading checkpoint
loaded checkpoint done.


In [12]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import numpy as np

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)


# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# start streaming video from webcam
video_stream()

# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # # collab webcam access
    
    width = float(frame.shape[0])  # float
    height = float(frame.shape[1])  # float
    fps = 30

    # timestamp = time.strftime("%Y_%m_%d_%H_%M_%S", current_time)
    # save_folder = osp.join(vis_folder, timestamp)
    # os.makedirs(save_folder, exist_ok=True)
    # if args.demo == "video":
    #     save_path = osp.join(save_folder, args.path.split("/")[-1])
    # else:
    #     save_path = osp.join(save_folder, "camera.mp4")
    # print(f"video save_path is {save_path}")
    # vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (int(width), int(height)))
    
    tracker = BYTETracker(args, frame_rate=30)
    timer = Timer()
    frame_id = 0
    results = []
    while True:
        if frame_id % 20 == 0:
            print('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))

            outputs, img_info = predictor.inference(frame, timer)
            
            if outputs[0] is not None:
                online_targets = tracker.update(outputs[0], [img_info['height'], img_info['width']], exp.test_size)
                online_tlwhs = []
                online_ids = []
                online_scores = []
                for t in online_targets:
                    tlwh = t.tlwh
                    tid = t.track_id
                    vertical = tlwh[2] / tlwh[3] > args.aspect_ratio_thresh
                    if tlwh[2] * tlwh[3] > args.min_box_area and not vertical:
                        online_tlwhs.append(tlwh)
                        online_ids.append(tid)
                        online_scores.append(t.score)
                        results.append(
                            f"{frame_id},{tid},{tlwh[0]:.2f},{tlwh[1]:.2f},{tlwh[2]:.2f},{tlwh[3]:.2f},{t.score:.2f},-1,-1,-1\n"
                        )
                timer.toc()
                online_im = plot_tracking(
                    img_info['raw_img'], online_tlwhs, online_ids, frame_id=frame_id + 1, fps=1. / timer.average_time
                )
            else:
                timer.toc()
                online_im = img_info['raw_img']

            # if args.save_result:
            #     vid_writer.write(online_im)

            cv2_imshow(online_im)
            
            ch = cv2.waitKey(1)
            if ch == 27 or ch == ord("q") or ch == ord("Q"):
                break
        
        else:
            break
        
        frame_id += 1

    # if args.save_result:
    #     res_file = osp.join(vis_folder, f"{timestamp}.txt")
    #     with open(res_file, 'w') as f:
    #         f.writelines(results)
    #     print(f"save results to {res_file}")

Output hidden; open in https://colab.research.google.com to view.